In [ ]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
from torch import nn
import matplotlib.pyplot as plt
import time

In [ ]:
transform = tv.transforms.Compose([
    tv.transforms.Resize((32,32)),
    tv.transforms.ToTensor()
])

In [ ]:
train_dataset = tv.datasets.FashionMNIST(root='.',
                                             train=True,
                                             download=True,
                                             transform=transform)
test_dataset = tv.datasets.FashionMNIST(root='.',
                                            train=False,
                                            download=True,
                                            transform=transform)

In [ ]:
BATCH_SIZE = 256
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [ ]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 50 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")

        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [ ]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(1024, 2048),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10))

In [ ]:
trainer = torch.optim.Adam(model.parameters(), lr=.001)

In [ ]:
train(model, train_iter, test_iter, trainer, num_epochs=35)

Step 0. time since epoch: 0.096. Train acc: 0.861. Train Loss: 0.556
Step 50. time since epoch: 4.581. Train acc: 0.871. Train Loss: 0.368
Step 100. time since epoch: 9.044. Train acc: 0.878. Train Loss: 0.343
--------------------
epoch 1, loss 0.3416, train acc 0.878, test acc 0.881, time 12.8 sec
Step 0. time since epoch: 0.089. Train acc: 0.930. Train Loss: 0.221
Step 50. time since epoch: 4.538. Train acc: 0.929. Train Loss: 0.187
Step 100. time since epoch: 9.102. Train acc: 0.932. Train Loss: 0.179
--------------------
epoch 2, loss 0.1798, train acc 0.932, test acc 0.877, time 12.9 sec
Step 0. time since epoch: 0.089. Train acc: 0.932. Train Loss: 0.178
Step 50. time since epoch: 4.583. Train acc: 0.937. Train Loss: 0.161
Step 100. time since epoch: 9.379. Train acc: 0.938. Train Loss: 0.159
--------------------
epoch 3, loss 0.1609, train acc 0.938, test acc 0.879, time 12.8 sec
Step 0. time since epoch: 0.093. Train acc: 0.938. Train Loss: 0.154
Step 50. time since epoch: 4.65

In [ ]:
evaluate_accuracy(test_iter, model.to(device))

0.8865